In [0]:
from pyspark.sql.functions import *
from delta.tables import *
df_air_data = spark.table("air_quality_project.air_quality_silver_data.hourly_air_quality")
#display(df_air_data)
df_air_daypart = (df_air_data.withColumn("day",to_date(col("timestamp"))).withColumn("daypart",
                       when(hour(col("timestamp")).between(0,5),"Night")
                       .when(hour(col("timestamp")).between(6,11),"Morning")
                       .when(hour(col("timestamp")).between(12,17),"Afternoon")
                       .otherwise("Evening")
                       ))
group_col = ["latitude","longitude","elevation","day","daypart"]
df_air_data_gold = df_air_daypart.groupBy(*group_col).agg(avg("pm10").alias("avg_pm10"),avg("pm2_5").alias("avg_pm2_5"))

#df_air_data_gold.show()
#df_air_data_gold.write.format("delta").mode("append").saveAsTable("")

target = DeltaTable.forName(spark,"air_quality_project.air_quality_gold_data.fact_air_quality_data")

#target_col = target.toDF().columns
insert_map = {
    "latitude": col("s.latitude"),
    "longitude": col("s.longitude"),
    "elevation": col("s.elevation"),
    "day": col("s.day"),
    "daypart": col("s.daypart"),
    "avg_pm10": col("s.avg_pm10"),
    "avg_pm2_5": col("s.avg_pm2_5"),
    "insertdate": current_timestamp(),   # Only set on insert
    "updateddate": None                  # Leave null on insert
}

(
    target.alias("t")
    .merge(
        df_air_data_gold.alias("s"),
        """t.latitude = s.latitude AND
        t.longitude = s.longitude AND
        t.elevation = s.elevation AND
        t.day = s.day AND
        t.daypart = s.daypart"""
    )
    .whenMatchedUpdate(set = {
         "avg_pm10": col("s.avg_pm10")
         ,"avg_pm2_5": col("s.avg_pm2_5")
         ,"updateddate": current_timestamp()
    })
    .whenNotMatchedInsert(values =insert_map)
    .execute()
)



In [0]:
%sql
select * from air_quality_project.air_quality_gold_data.fact_air_quality_data